In [ ]:
!nvidia-smi

Wed Jan 27 07:29:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/vin/NLP/nmt_attention2'

/content/drive/MyDrive/vin/NLP/nmt_attention2


In [ ]:
!pip install tensorflow-addons==0.11.2

     |████████████████████████████████| 1.1MB 8.5MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import pickle
import numpy as np
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [ ]:
def preprocess_sentence(s):
    s = s.lower()
    s = s.strip()
    s = '<s> ' + s + ' </s>'
    return s

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    ##________ LSTM layer in Encoder ------- ##
    self.lstm_layer = tf.keras.layers.LSTM(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')



  def call(self, x, hidden):
    x = self.embedding(x)
    output, h, c = self.lstm_layer(x, initial_state = hidden)
    return output, h, c

  def initialize_hidden_state(self):
    return [tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))]

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz, max_length_en, max_length_vi, attention_type='luong'):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.attention_type = attention_type
    self.max_length_en = max_length_en
    self.max_length_vi = max_length_vi

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)

    self.fc = tf.keras.layers.Dense(vocab_size)

    self.decoder_rnn_cell = tf.keras.layers.LSTMCell(self.dec_units)

    self.sampler = tfa.seq2seq.sampler.TrainingSampler()

    self.attention_mechanism = self.build_attention_mechanism(self.dec_units, 
                                                              None, self.batch_sz*[self.max_length_en], self.attention_type)

    self.rnn_cell = self.build_rnn_cell(batch_sz)

    self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler=self.sampler, output_layer=self.fc)


  def build_rnn_cell(self, batch_sz):
    rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnn_cell, 
                                  self.attention_mechanism, attention_layer_size=self.dec_units)
    return rnn_cell

  def build_attention_mechanism(self, dec_units, memory, memory_sequence_length, attention_type='luong'):
   
    if(attention_type=='bahdanau'):
      return tfa.seq2seq.BahdanauAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)
    else:
      return tfa.seq2seq.LuongAttention(units=dec_units, memory=memory, memory_sequence_length=memory_sequence_length)

  def build_initial_state(self, batch_sz, encoder_state, Dtype):
    decoder_initial_state = self.rnn_cell.get_initial_state(batch_size=batch_sz, dtype=Dtype)
    decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state)
    return decoder_initial_state


  def call(self, inputs, initial_state):
    x = self.embedding(inputs)
    outputs, _, _ = self.decoder(x, initial_state=initial_state, sequence_length=self.batch_sz*[self.max_length_vi-1])
    return outputs

In [ ]:
class Model:
    def __init__(self):
        self.decoder = None
        self.encoder = None
        self.optimizer = None
        self.en_tokenizer = None
        self.vi_tokenizer = None
        self.loaded_model = False
        self.batch_size = 1
        self.embedding_dim = 256
        self.units = 1024
        self.vocab_en_size = None
        self.vocab_vi_size = None
        self.attention = None
    
    def load_model(self, checkpoint_dir, dataset_file):
        with open(dataset_file, 'rb') as f:
            data = pickle.load(f)
            self.en_tokenizer = data['en_tokenizer']
            self.vi_tokenizer = data['vi_tokenizer']
            self.max_length_en = data['max_length_en']
            self.max_length_vi = data['max_length_vi']
            self.attention = data['attention']
            self.en_example = data['en_example']
            self.vi_example = data['vi_example']
            self.vocab_en_size = len(self.en_tokenizer.word_index)+1
            self.vocab_vi_size = len(self.vi_tokenizer.word_index)+1

        self.encoder = Encoder(self.vocab_en_size, self.embedding_dim, self.units, self.batch_size)
        self.decoder = Decoder(self.vocab_vi_size, self.embedding_dim, self.units, self.batch_size, self.max_length_en, self.max_length_vi, self.attention)
        self.optimizer = tf.keras.optimizers.Adam()
        
        self.re_train(tf.convert_to_tensor([self.en_example]), tf.convert_to_tensor([self.vi_example]))

        checkpoint = tf.train.Checkpoint(optimizer=self.optimizer,
                                    encoder=self.encoder,
                                    decoder=self.decoder)

        status = checkpoint.restore(checkpoint_dir)
        self.loaded_model = True
        return True
    
    def evaluate_sentence(self, sentence):
        sentence = preprocess_sentence(sentence)

        inputs = [self.en_tokenizer.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=self.max_length_en,
                                                                padding='post')
        inputs = tf.convert_to_tensor(inputs)
        inference_batch_size = inputs.shape[0]
        result = ''

        enc_start_state = [tf.zeros((inference_batch_size, self.units)), tf.zeros((inference_batch_size,self.units))]
        enc_out, enc_h, enc_c = self.encoder(inputs, enc_start_state)

        dec_h = enc_h
        dec_c = enc_c

        start_tokens = tf.fill([inference_batch_size], self.vi_tokenizer.word_index['<s>'])
        end_token = self.vi_tokenizer.word_index['</s>']

        greedy_sampler = tfa.seq2seq.GreedyEmbeddingSampler()

        decoder_instance = tfa.seq2seq.BasicDecoder(cell=self.decoder.rnn_cell, sampler=greedy_sampler, output_layer=self.decoder.fc)
        self.decoder.attention_mechanism.setup_memory(enc_out)

        decoder_initial_state = self.decoder.build_initial_state(inference_batch_size, [enc_h, enc_c], tf.float32)

        decoder_embedding_matrix = self.decoder.embedding.variables[0]

        outputs, _, _ = decoder_instance(decoder_embedding_matrix, start_tokens = start_tokens, end_token = end_token, initial_state=decoder_initial_state)
        return outputs.sample_id.numpy()

    def basic_translate(self, sentence):
        result = self.evaluate_sentence(sentence)
        result = self.vi_tokenizer.sequences_to_texts(result)
        return result


    # BeamSearchDecoder
    def beam_evaluate_sentence(self, sentence, beam_width=5):
        sentence = preprocess_sentence(sentence)

        inputs = [self.en_tokenizer.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=self.max_length_en,
                                                                padding='post')
        inputs = tf.convert_to_tensor(inputs)
        inference_batch_size = inputs.shape[0]
        result = ''

        enc_start_state = [tf.zeros((inference_batch_size, self.units)), tf.zeros((inference_batch_size, self.units))]
        enc_out, enc_h, enc_c = self.encoder(inputs, enc_start_state)

        dec_h = enc_h
        dec_c = enc_c

        start_tokens = tf.fill([inference_batch_size], self.vi_tokenizer.word_index['<s>'])
        end_token = self.vi_tokenizer.word_index['</s>']

        enc_out = tfa.seq2seq.tile_batch(enc_out, multiplier=beam_width)
        self.decoder.attention_mechanism.setup_memory(enc_out)

        hidden_state = tfa.seq2seq.tile_batch([enc_h, enc_c], multiplier=beam_width)
        decoder_initial_state = self.decoder.rnn_cell.get_initial_state(batch_size=beam_width*inference_batch_size, dtype=tf.float32)
        decoder_initial_state = decoder_initial_state.clone(cell_state=hidden_state)

        decoder_instance = tfa.seq2seq.BeamSearchDecoder(self.decoder.rnn_cell,beam_width=beam_width, output_layer=self.decoder.fc)
        decoder_embedding_matrix = self.decoder.embedding.variables[0]

        outputs, final_state, sequence_lengths = decoder_instance(decoder_embedding_matrix, start_tokens=start_tokens, end_token=end_token, initial_state=decoder_initial_state)

        final_outputs = tf.transpose(outputs.predicted_ids, perm=(0,2,1))
        beam_scores = tf.transpose(outputs.beam_search_decoder_output.scores, perm=(0,2,1))

        return final_outputs.numpy(), beam_scores.numpy()

    def beam_translate(self, sentence):
        result, beam_scores = self.beam_evaluate_sentence(sentence)
        for beam, score in zip(result, beam_scores):
            output = self.vi_tokenizer.sequences_to_texts(beam)
            output = [a[:a.index('</s>')] for a in output]
            beam_score = [a.sum() for a in score]
            return output[0]
        
    def loss_function(self, real, pred):
        cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
        loss = cross_entropy(y_true=real, y_pred=pred)
        mask = tf.logical_not(tf.math.equal(real,0))
        mask = tf.cast(mask, dtype=loss.dtype)  
        loss = mask* loss
        loss = tf.reduce_mean(loss)
        return loss

    @tf.function
    def train_step(self, inp, targ, enc_hidden):
        loss = 0
        with tf.GradientTape() as tape:
            enc_output, enc_h, enc_c = self.encoder(inp, enc_hidden)

            dec_input = targ[ : , :-1 ]
            real = targ[ : , 1: ]      

            self.decoder.attention_mechanism.setup_memory(enc_output)

            decoder_initial_state = self.decoder.build_initial_state(self.batch_size, [enc_h, enc_c], tf.float32)
            pred = self.decoder(dec_input, decoder_initial_state)
            logits = pred.rnn_output
            loss = self.loss_function(real, logits)

        variables = self.encoder.trainable_variables + self.decoder.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))

        return loss

    def re_train(self, inp, targ):
        enc_hidden = self.encoder.initialize_hidden_state()
        batch_loss = self.train_step(inp, targ, enc_hidden)

    def translate(self, input_sentence):
        attention_plot = np.zeros((self.max_length_vi, self.max_length_en))
        sentence = preprocess_sentence(input_sentence)

        inputs = [self.en_tokenizer.word_index[i] for i in sentence.split(' ')]
        inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=self.max_length_en,
                                                                padding='post')
        inputs = tf.convert_to_tensor(inputs)

        result = ''

        hidden = [tf.zeros((1, self.units))]
        enc_out, enc_hidden = self.encoder(inputs, hidden)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([self.vi_tokenizer.word_index['<s>']], 0)

        for t in range(self.max_length_vi):
            predictions, dec_hidden, attention_weights = self.decoder(dec_input,
                                                                dec_hidden,
                                                                enc_out)

            attention_weights = tf.reshape(attention_weights, (-1, ))
            attention_plot[t] = attention_weights.numpy()

            predicted_id = tf.argmax(predictions[0]).numpy()

            result += self.vi_tokenizer.index_word[predicted_id] + ' '

            if self.vi_tokenizer.index_word[predicted_id] == '</s>':
                return result, sentence, attention_plot

            dec_input = tf.expand_dims([predicted_id], 0)
        return result, sentence, attention_plot

In [ ]:
en_val_path = 'processed_text_data/dev/tst2012.en'
vi_val_path = 'processed_text_data/dev/tst2012.vi'

In [ ]:
import gc
with open(en_val_path, 'r') as f:
  text_en_dev = f.read().split("\n")

with open(vi_val_path, 'r') as f:
  text_vi_dev = f.read().split("\n")

min_len = 3
max_len = 50
en_dev_arr = []
vi_dev_arr = []
for en_sent, vi_sent in zip(text_en_dev, text_vi_dev):
  en_len = len(en_sent.strip().split(" "))
  vi_len = len(vi_sent.strip().split(" "))
  if en_len < min_len or en_len > max_len or vi_len < min_len or vi_len > max_len:
    continue
  en_dev_arr.append(en_sent)
  vi_dev_arr.append(vi_sent)

In [ ]:
reference_arr = [sent.replace('_', ' ') for sent in vi_dev_arr]
reference = "\n".join(reference_arr)
def create_candidate(model, ref_path, can_path):
  candidate_arr = []
  for sent in tqdm(en_dev_arr):
    result = model.basic_translate(sent)
    candidate_arr.append(result)
  candidate_arr = [sent[0].replace(' </s>', '').replace('_', ' ') for sent in candidate_arr]
  candidate = "\n".join(candidate_arr)
  with open(ref_path, 'w') as f:
    f.write(reference)
  with open(can_path, 'w') as f:
    f.write(candidate)

In [ ]:
# !wget https://raw.githubusercontent.com/OpenNMT/OpenNMT-py/master/tools/multi-bleu.perl

In [ ]:
cp_arr = []
for i in range(3, 32, 2):
  cp_arr.append('ckpt-'+str(i))

In [ ]:
import os
for cp in cp_arr:
  re_path = os.path.join('result/luong/beam/',cp)
  try:  
    os.makedirs(re_path)  
  except OSError as error:
    print(error)

[Errno 17] File exists: 'result/luong/beam/ckpt-3'
[Errno 17] File exists: 'result/luong/beam/ckpt-5'
[Errno 17] File exists: 'result/luong/beam/ckpt-7'
[Errno 17] File exists: 'result/luong/beam/ckpt-9'
[Errno 17] File exists: 'result/luong/beam/ckpt-11'
[Errno 17] File exists: 'result/luong/beam/ckpt-13'
[Errno 17] File exists: 'result/luong/beam/ckpt-15'
[Errno 17] File exists: 'result/luong/beam/ckpt-17'
[Errno 17] File exists: 'result/luong/beam/ckpt-19'
[Errno 17] File exists: 'result/luong/beam/ckpt-21'
[Errno 17] File exists: 'result/luong/beam/ckpt-23'
[Errno 17] File exists: 'result/luong/beam/ckpt-25'
[Errno 17] File exists: 'result/luong/beam/ckpt-27'
[Errno 17] File exists: 'result/luong/beam/ckpt-29'
[Errno 17] File exists: 'result/luong/beam/ckpt-31'


In [ ]:
for cp in cp_arr:
  checkpoint_dir = os.path.join('checkpoints/luong_cp/',cp)
  re_path = os.path.join('result/luong/beam/',cp)
  infor = 'information/infor_luong.pickle'
  luong_model = Model()
  luong_model.load_model(checkpoint_dir=checkpoint_dir, dataset_file=infor)
  ref_path = os.path.join(re_path, 'reference.vi')
  can_path = os.path.join(re_path, 'candidate.vi')
  create_candidate_beam_search(luong_model, ref_path, can_path)
  print('checkpoint {}'.format(cp), ':')
  !perl multi-bleu.perl $ref_path < $can_path

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/resource_loader.py:103: UserWarning: You are currently using TensorFlow 2.4.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops against TensorFlow 2.2.0, and there are no compatibility guarantees between the two versions. 
This means that you might get segfaults when loading the custom op, or other kind of low-level errors.
 If you do, do not file an issue on Github. This is a known limitation.

It might help you to fallback to pure Python ops with TF_ADDONS_PY_OPS . To do that, see https://github.com/tensorflow/addons#gpucpu-custom-ops 

You can also change the TensorFlow version installed on your system. You would need a TensorFlow version equal to or above 2.2.0 and strictly below 2.3.0.
 Note that nightly versions of TensorFlow, as well as non-pip TensorFlow like `conda install tensorflow` or compiled from source are not supported.

The last solution i


checkpoint ckpt-3 :
BLEU = 21.55, 56.6/30.7/17.4/9.8 (BP=0.923, ratio=0.926, hyp_len=29775, ref_len=32157)



checkpoint ckpt-5 :
BLEU = 23.25, 57.8/32.0/18.6/10.7 (BP=0.943, ratio=0.944, hyp_len=30372, ref_len=32157)



checkpoint ckpt-7 :
BLEU = 23.37, 57.9/32.3/18.8/10.9 (BP=0.939, ratio=0.941, hyp_len=30250, ref_len=32157)



checkpoint ckpt-9 :
BLEU = 22.56, 57.3/31.4/18.1/10.3 (BP=0.937, ratio=0.938, hyp_len=30179, ref_len=32157)


...


checkpoint ckpt-19 :
BLEU = 20.71, 54.8/29.1/16.2/8.9 (BP=0.945, ratio=0.947, hyp_len=30438, ref_len=32157)



checkpoint ckpt-21 :
BLEU = 20.17, 53.8/28.2/15.6/8.8 (BP=0.945, ratio=0.946, hyp_len=30429, ref_len=32157)



checkpoint ckpt-23 :
BLEU = 19.47, 53.6/27.9/15.3/8.3 (BP=0.933, ratio=0.935, hyp_len=30059, ref_len=32157)



checkpoint ckpt-25 :
BLEU = 19.60, 53.5/27.8/15.1/8.3 (BP=0.944, ratio=0.945, hyp_len=30390, ref_len=32157)


KeyboardInterrupt: ignored

In [ ]:
checkpoint_dir = os.path.join('checkpoints/luong_cp/','ckpt-3')
re_path = os.path.join('result/luong/gready/','cpkt-3')
infor = 'information/infor_luong.pickle'
luong_model = Model()
luong_model.load_model(checkpoint_dir=checkpoint_dir, dataset_file=infor)

True

In [ ]:
cp_arr = []
for i in range(1, 3):
  cp_arr.append('ckpt-'+str(i))

In [ ]:
import os
for cp in cp_arr:
  re_path = os.path.join('result/luong/gready/',cp)
  try:  
    os.makedirs(re_path)  
  except OSError as error:  
    print(error)

[Errno 17] File exists: 'result/luong/gready/ckpt-1'


## Đánh giá trên tập Test

In [ ]:
import os
checkpoint_dir = os.path.join('checkpoints/luong_cp/','ckpt-7')
re_path = os.path.join('result/luong/gready/','cpkt-7')
infor = 'information/infor_luong.pickle'
luong_model = Model()
luong_model.load_model(checkpoint_dir=checkpoint_dir, dataset_file=infor)

True

In [ ]:
def create_candidate_beam_search(model, ref_path, can_path):
  candidate_arr = []
  for sent in tqdm(en_dev_arr):
    result = model.beam_translate(sent)
    
    candidate_arr.append(result)

  candidate_arr = [sent.replace(' </s>', '').replace('_', ' ') for sent in candidate_arr]
  candidate = "\n".join(candidate_arr)
  with open(ref_path, 'a+') as f:
    f.write(reference)
  with open(can_path, 'a+') as f:
    f.write(candidate)

In [ ]:
import gc
with open('processed_text_data/test/tst2013.en', 'r') as f:
  text_en_dev = f.read().split("\n")

with open('processed_text_data/test/tst2013.vi', 'r') as f:
  text_vi_dev = f.read().split("\n")

min_len = 3
max_len = 50
en_dev_arr = []
vi_dev_arr = []
for en_sent, vi_sent in zip(text_en_dev, text_vi_dev):
  en_len = len(en_sent.strip().split(" "))
  vi_len = len(vi_sent.strip().split(" "))
  if en_len < min_len or en_len > max_len or vi_len < min_len or vi_len > max_len:
    continue
  en_dev_arr.append(en_sent)
  vi_dev_arr.append(vi_sent)

reference_arr = [sent.replace('_', ' ') for sent in vi_dev_arr]
reference = "\n".join(reference_arr)

In [ ]:
vi_dev_arr = vi_dev_arr[527:647]
en_dev_arr = en_dev_arr[527:647]
reference_arr = [sent.replace('_', ' ') for sent in vi_dev_arr]
reference = "\n".join(reference_arr)

In [ ]:
create_candidate_beam_search(luong_model, 'result/luong/test/reference.vi', 'result/luong/test/candidate.vi')

In [ ]:
!perl multi-bleu.perl 'result/luong/test/reference.vi' < 'result/luong/test/candidate.vi'

BLEU = 24.41, 59.7/33.4/19.6/11.8 (BP=0.937, ratio=0.939, hyp_len=15043, ref_len=16024)
